In [3]:
# | default_exp agent.planner.models

In [2]:
# | export
from typing import Dict, List, Any, Optional, Union
from datetime import datetime
from enum import Enum
from pydantic import BaseModel, Field, validator

In [3]:
# | export
class ExecutionStatus(str, Enum):
    PENDING = "pending"
    IN_PROGRESS = "in_progress"
    SUCCESS = "success"
    PARTIAL_SUCCESS = "partial_success"
    FAILED = "failed"
    RETRYING = "retrying"

class ExecutionMode(Enum):
    SEQUENTIAL = "sequential"
    PARALLEL = "parallel"

class Resources(BaseModel):
    previous: List[str] = Field(default_factory=list, description="Resources created in previous steps")
    required: List[str] = Field(default_factory=list, description="Resources helpful or required for this/current next step")
    
class ActionStep(BaseModel):
    step: int = Field(..., description="The step number")
    purpose: str = Field(..., description="The purpose of this step")
    sub_steps: List[str] = Field(default_factory=list, description="Sub-steps if any")
    introspect_after: bool = Field(True, description="Whether to call introspect after this step")
    system_prompt: str = Field(..., description="Self-sufficient system prompt for this subtask")
    user_prompt: str = Field(..., description="Self-sufficient user prompt for this subtask")
    introspect_prompt: str = Field(..., description="Template prompt for introspect")
    execution_mode: ExecutionMode = Field(ExecutionMode.SEQUENTIAL, description="Sequential or parallel execution")
    
class Task(BaseModel):
    id: str = Field(..., description="Unique task ID, e.g., T001")
    name: str = Field(..., description="Concise task name")
    description: str = Field(..., description="Detailed task description")
    dependencies: List[str] = Field(default_factory=list, description="List of dependent task IDs")
    actions: List[ActionStep] = Field(default_factory=list, description="List of action steps")
    success_criteria: str = Field(..., description="Criteria for task success")
    expected_outputs: List[str] = Field(default_factory=list, description="Expected output artifacts")
    resources: Resources = Field(default_factory=Resources, description="Overall resources for the task")
    potential_options: List[str] = Field(default_factory=list, description="List of potential options for decision making")
    needs_debate: bool = Field(False, description="Whether this task requires debating options")

class ActionResult(BaseModel):
    action_id: str = Field(..., description="Unique action identifier")
    tool_used: str = Field(..., description="Tool that was executed")
    result: Any = Field(..., description="Raw result from tool execution")
    execution_time: float = Field(..., description="Time taken in seconds")
    status: ExecutionStatus = Field(..., description="Execution status")
    error_message: Optional[str] = Field(None, description="Error message if failed")
    artifacts_created: List[str] = Field(default_factory=list, description="Files/artifacts created")

class IntrospectionResult(BaseModel):
    success: bool = Field(..., description="Whether introspection passed")
    score: float = Field(..., description="Performance score 0-10")
    feedback: str = Field(..., description="Detailed feedback")
    next_action: str = Field(..., description="proceed, retry, or escalate")
    recommendations: List[str] = Field(default_factory=list, description="Improvement suggestions")

class TaskExecutionResult(BaseModel):
    task_id: str = Field(..., description="Task identifier")
    task_name: str = Field(..., description="Task name")
    status: ExecutionStatus = Field(..., description="Overall task status")
    actions_executed: List[ActionResult] = Field(default_factory=list, description="ActionStep results")
    introspection_results: List[IntrospectionResult] = Field(default_factory=list, description="Introspection results")
    artifacts_created: List[str] = Field(default_factory=list, description="All artifacts from this task")
    execution_time: float = Field(..., description="Total task execution time")
    timestamp: datetime = Field(default_factory=datetime.now, description="Execution timestamp")
    success_criteria_met: bool = Field(..., description="Whether success criteria were met")

class ProjectContext(BaseModel):
    original_request: str = Field(..., description="Original user request")
    execution_history: List[TaskExecutionResult] = Field(default_factory=list, description="Task execution history")
    current_artifacts: List[str] = Field(default_factory=list, description="All project artifacts")
    total_tasks_completed: int = Field(0, description="Number of completed tasks")
    project_status: ExecutionStatus = Field(ExecutionStatus.PENDING, description="Overall project status")
    failed_tasks: List[str] = Field(default_factory=list, description="List of failed task IDs")

class ProjectBreakdown(BaseModel):
    title: str = Field(..., description="Project title based on user request")
    project_summary: str = Field(..., description="High-level project summary")
    phases: List[str] = Field(..., description="Major project phases")
    key_deliverables: List[str] = Field(..., description="Key deliverables expected")
    technical_approach: str = Field(..., description="Technical approach and architecture")

